In [1]:
import random
#random.seed(99)
import pandas as pd
import numpy as np
import dask.dataframe as dd
import seaborn as sns
import dask.datasets
from dask_sql import Context
import datetime
import time
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

pd.options.mode.chained_assignment = None

# Подсчет количества пользователей и запросов по категории на одном файле

In [2]:
def agg_df(df, year):
    start_dt = datetime.datetime.now()
    print('Start:', start_dt)
    df1 = df[['user', 'rubricName', 'date']]
    
    try:
        pd_df1_req_cnt = pd.read_excel(f'df1_req_cnt_{year}.xlsx')
        pd_df1_user_cnt = pd.read_excel(f'df1_user_cnt_{year}.xlsx')
        df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_index=True, right_index=True)
    except:
        df1_user_cnt = df1.groupby('rubricName').user.nunique().compute()
        df1_req_cnt = df1.groupby('rubricName').date.count().compute()
        df1_req_cnt.to_excel(f'df1_req_cnt_{year}.xlsx')
        df1_user_cnt.to_excel(f'df1_user_cnt_{year}.xlsx')
        pd_df1_req_cnt = pd.DataFrame(df1_req_cnt)
        pd_df1_user_cnt = pd.DataFrame(df1_user_cnt)
        df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_on='rubricName', right_on='rubricName')
    
    df1_agg = df1_agg.sort_values('date', ascending=False)
    df1_agg['rubricName'] = df1_agg['rubricName_x']
    df1_agg = df1_agg.set_index('rubricName')[['date', 'user']].fillna(0).astype(int)
    df1_agg.to_excel(f'df1_agg_{year}.xlsx', index=False)
    end_dt = datetime.datetime.now()
    print('End:', end_dt)
    print('Duration:', end_dt-start_dt)
    return df1_agg

### Москва

In [3]:
df20_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2020_969792.csv", dtype={'fl': 'object'})
df20_agg = agg_df(df20_raw, 2020)
df21_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2021_970748.csv", dtype={'fl': 'object'})
df21_agg = agg_df(df21_raw, 2021)
df22_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2022_970753.csv", dtype={'fl': 'object'})
df22_agg = agg_df(df22_raw, 2022)
df23_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2023_970754.csv", dtype={'fl': 'object'})
df23_agg = agg_df(df23_raw, 2023)

Start: 2023-03-13 16:27:50.341337
End: 2023-03-13 16:27:53.322987
Duration: 0:00:02.981650
Start: 2023-03-13 16:27:53.373402
End: 2023-03-13 16:27:53.922772
Duration: 0:00:00.549370
Start: 2023-03-13 16:27:53.970086
End: 2023-03-13 16:27:54.445928
Duration: 0:00:00.475842
Start: 2023-03-13 16:27:54.524908
End: 2023-03-13 16:27:55.025557
Duration: 0:00:00.500649


### СПБ

In [21]:
spb20_raw = dd.read_csv(r"C:\Users\Mamed\Downloads\СПБ_2020_974783.csv")
spb20_agg = agg_df(spb20_raw, 'spb_2020')

Start: 2023-03-13 17:56:43.702246
End: 2023-03-13 17:56:43.902273
Duration: 0:00:00.200027


In [ ]:
spb21_raw = dd.read_csv(r"C:\Users\Mamed\Downloads\СПБ_2021_974784.csv")
spb21_agg = agg_df(spb21_raw, 'spb_2021')

# Подсчет количества пользователей по всем файлам

### Все категории

In [70]:
#df_all = dd.concat([df20_raw,df21_raw,df22_raw,df23_raw])

#df_all.to_parquet('df_all')

df_all = dd.read_parquet('df_all')

users_itogo = df_all.groupby('rubricName').user.nunique().compute()

users_itogo_result = df_all.user.nunique().compute()

df20_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2020_969792.csv", dtype={'fl': 'object'})
df21_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2021_970748.csv", dtype={'fl': 'object'})
df22_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2022_970753.csv", dtype={'fl': 'object'})
df23_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2023_970754.csv", dtype={'fl': 'object'})

df20_result_cnt = df20_raw.user.nunique().compute()
df21_result_cnt = df21_raw.user.nunique().compute()
df22_result_cnt = df22_raw.user.nunique().compute()
df23_result_cnt = df23_raw.user.nunique().compute()

print(users_itogo_result, 
df20_result_cnt,
df21_result_cnt,
df22_result_cnt,
df23_result_cnt,)

users_itogo.to_excel('users_itogo.xlsx')

5512385 1926180 1624863 1920044 1947122


### Часть категорий

In [80]:
rubrics = pd.read_excel(r"C:\Users\Mamed\Downloads\Свод_2ГИС_Москва_230311_0145.xlsx")[['Рубрика']]
df_all = dd.read_parquet('df_all')
df_filtered = df_all.merge(rubrics, left_on='rubricName', right_on='Рубрика')

df_filtered.to_parquet('df_filtered')

users_filtered_itogo = df_filtered.groupby('rubricName').user.nunique().compute()
users_filtered_itogo_result = df_filtered.user.nunique().compute()
users_filtered_itogo.to_excel('users_filtered_itogo.xlsx')

df20_filtered = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2020_969792.csv", dtype={'fl': 'object'}).merge(rubrics, left_on='rubricName', right_on='Рубрика')
df21_filtered = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2021_970748.csv", dtype={'fl': 'object'}).merge(rubrics, left_on='rubricName', right_on='Рубрика')
df22_filtered = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2022_970753.csv", dtype={'fl': 'object'}).merge(rubrics, left_on='rubricName', right_on='Рубрика')
df23_filtered = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2023_970754.csv", dtype={'fl': 'object'}).merge(rubrics, left_on='rubricName', right_on='Рубрика')

df20_filtered_result_cnt = df20_filtered.user.nunique().compute()
df21_filtered_result_cnt = df21_filtered.user.nunique().compute()
df22_filtered_result_cnt = df22_filtered.user.nunique().compute()
df23_filtered_result_cnt = df23_filtered.user.nunique().compute()

print(users_filtered_itogo_result, 
df20_filtered_result_cnt,
df21_filtered_result_cnt,
df22_filtered_result_cnt,
df23_filtered_result_cnt,)

4459161 1627743 1305787 1536122 1536886


# Алгоритм сокращения топ-мест в рейтингах для категорий

In [955]:
def row_number(df, col, sort_col, desc=True, filter_df=True, index=30, filter_zero=False):
    if filter_df:
        rows_to_count = df[df[col]].sort_values(sort_col,ascending=not desc).loc[:,[col]].index[:index]
    elif filter_zero:
        rows_to_count = df[df[col]!=0].sort_values(sort_col,ascending=not desc).loc[:,[col]].index[:index]
    else:
        rows_to_count = df.sort_values(sort_col,ascending=not desc).loc[:,[col]].index[:index]
        df_shape = df.shape
    rows_to_zero = [i for i in df.index if i not in rows_to_count] 
    n = 0
    for r0 in rows_to_zero:
        df.loc[r0, col] = 0
    for r in rows_to_count:
        n += 1
        df.loc[r, col] = n
    
    return df

In [956]:
def count_ratings(df):
    df_user_max = df.loc[:,['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']].max(axis=1)
    df_user_min = df.loc[:,['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']].min(axis=1)

    df['Рейтинг по пользователям 2020'] = df_user_max == df.loc[:,['Пользователи 2020']].iloc[:,0]
    df['Рейтинг по пользователям 2021'] = df_user_max == df.loc[:,['Пользователи 2021']].iloc[:,0]
    df['Рейтинг по пользователям 2022'] = df_user_max == df.loc[:,['Пользователи 2022']].iloc[:,0]
    df['Рейтинг по пользователям 2023'] = df_user_max == df.loc[:,['Пользователи 2023']].iloc[:,0]

    df['Антирейтинг по пользователям 2020'] = df_user_min == df.loc[:,['Пользователи 2020']].iloc[:,0]
    df['Антирейтинг по пользователям 2021'] = df_user_min == df.loc[:,['Пользователи 2021']].iloc[:,0]
    df['Антирейтинг по пользователям 2022'] = df_user_min == df.loc[:,['Пользователи 2022']].iloc[:,0]
    df['Антирейтинг по пользователям 2023'] = df_user_min == df.loc[:,['Пользователи 2023']].iloc[:,0]

    df['Рейтинг постоянного\nроста пользователей'] = (df['Пользователи 2020'] < df['Пользователи 2021']
        & (df['Пользователи 2021'] < df['Пользователи 2022'])
        & (df['Пользователи 2022'] < df['Пользователи 2023']))

    df['Рейтинг постоянного\nпадения пользователей'] = ((df['Пользователи 2020'] > df['Пользователи 2021']) 
        & (df['Пользователи 2021'] > df['Пользователи 2022'])
        & (df['Пользователи 2022'] > df['Пользователи 2023']))

    df['Рейтинг по пользователям ВНВН'] = ((df['Пользователи 2020'] > df['Пользователи 2021']) 
        & (df['Пользователи 2021'] < df['Пользователи 2022'])
        & (df['Пользователи 2022'] > df['Пользователи 2023']))

    df['Рейтинг по пользователям НВНВ'] = ((df['Пользователи 2020'] < df['Пользователи 2021']) 
        & (df['Пользователи 2021'] > df['Пользователи 2022'])
        & (df['Пользователи 2022'] < df['Пользователи 2023']))

    df['Рейтинг стабильности\nчисла пользователей'] = abs(df_user_min / df_user_max - 1)

    df = row_number(df, 'Рейтинг по пользователям 2020', 'Пользователи 2020')
    df = row_number(df, 'Рейтинг по пользователям 2021', 'Пользователи 2021')
    df = row_number(df, 'Рейтинг по пользователям 2022', 'Пользователи 2022')
    df = row_number(df, 'Рейтинг по пользователям 2023', 'Пользователи 2023')
    df = row_number(df, 'Рейтинг по пользователям, итого', 'Пользователи, итого', filter_df=False)

    df = row_number(df, 'Антирейтинг по пользователям 2020', 'Пользователи 2020', desc=False)
    df = row_number(df, 'Антирейтинг по пользователям 2021', 'Пользователи 2021', desc=False)
    df = row_number(df, 'Антирейтинг по пользователям 2022', 'Пользователи 2022', desc=False)
    df = row_number(df, 'Антирейтинг по пользователям 2023', 'Пользователи 2023', desc=False)

    df = row_number(df, 'Рейтинг постоянного\nроста пользователей', 'Пользователи, итого')
    df = row_number(df, 'Рейтинг постоянного\nпадения пользователей', 'Пользователи, итого')
    df = row_number(df, 'Рейтинг по пользователям ВНВН', 'Пользователи 2023')
    df = row_number(df, 'Рейтинг по пользователям НВНВ', 'Пользователи 2023')
    df = row_number(df, 'Рейтинг стабильности\nчисла пользователей', 'Пользователи, итого', filter_df=False)

    df_req_max = df.loc[:,['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']].max(axis=1)
    df_req_min = df.loc[:,['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']].min(axis=1)

    df['Рейтинг по запросам 2020'] = df_req_max == df.loc[:,['Запросы 2020']].iloc[:,0]
    df['Рейтинг по запросам 2021'] = df_req_max == df.loc[:,['Запросы 2021']].iloc[:,0]
    df['Рейтинг по запросам 2022'] = df_req_max == df.loc[:,['Запросы 2022']].iloc[:,0]
    df['Рейтинг по запросам 2023'] = df_req_max == df.loc[:,['Запросы 2023']].iloc[:,0]
    df['Рейтинг по запросам, итого'] = df_req_max == df.loc[:,['Запросы, итого']].iloc[:,0]

    df['Антирейтинг по запросам 2020'] = df_req_min == df.loc[:,['Запросы 2020']].iloc[:,0]
    df['Антирейтинг по запросам 2021'] = df_req_min == df.loc[:,['Запросы 2021']].iloc[:,0]
    df['Антирейтинг по запросам 2022'] = df_req_min == df.loc[:,['Запросы 2022']].iloc[:,0]
    df['Антирейтинг по запросам 2023'] = df_req_min == df.loc[:,['Запросы 2023']].iloc[:,0]

    df['Рейтинг постоянного\nроста запросов'] = ((df['Запросы 2020'] < df['Запросы 2021']) 
        & (df['Запросы 2021'] < df['Запросы 2022'])
        & (df['Запросы 2022'] < df['Запросы 2023']))

    df['Рейтинг постоянного\nпадения запросов'] = ((df['Запросы 2020'] > df['Запросы 2021']) 
        & (df['Запросы 2021'] > df['Запросы 2022'])
        & (df['Запросы 2022'] > df['Запросы 2023']))

    df['Рейтинг по запросам ВНВН'] = ((df['Пользователи 2020'] > df['Пользователи 2021']) 
        & (df['Пользователи 2021'] < df['Пользователи 2022'])
        & (df['Пользователи 2022'] > df['Пользователи 2023']))

    df['Рейтинг по запросам НВНВ'] = ((df['Запросы 2020'] < df['Запросы 2021']) 
        & (df['Запросы 2021'] > df['Запросы 2022'])
        & (df['Запросы 2022'] < df['Запросы 2023']))

    df['Рейтинг стабильности\nчисла запросов'] = abs(df_req_min / df_req_max - 1)

    from scipy.stats.stats import pearsonr

    x = df[['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']]
    y = df[['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']]
    df = df.fillna(0)
    for row in df.index:
        df.loc[row, 'Антирейтинг схожести динамики\nпользователей и запросов'] = scipy.stats.pearsonr(x.loc[row], y.loc[row])[0]
    df['Рейтинг схожести динамики\nпользователей и запросов'] = df['Антирейтинг схожести динамики\nпользователей и запросов']


    df = row_number(df, 'Рейтинг по запросам 2020', 'Запросы 2020')
    df = row_number(df, 'Рейтинг по запросам 2021', 'Запросы 2021')
    df = row_number(df, 'Рейтинг по запросам 2022', 'Запросы 2022')
    df = row_number(df, 'Рейтинг по запросам 2023', 'Запросы 2023')
    df = row_number(df, 'Рейтинг по запросам, итого', 'Запросы, итого', filter_df=False)

    df = row_number(df, 'Антирейтинг по запросам 2020', 'Запросы 2020', desc=False)
    df = row_number(df, 'Антирейтинг по запросам 2021', 'Запросы 2021', desc=False)
    df = row_number(df, 'Антирейтинг по запросам 2022', 'Запросы 2022', desc=False)
    df = row_number(df, 'Антирейтинг по запросам 2023', 'Запросы 2023', desc=False)

    df = row_number(df, 'Рейтинг постоянного\nроста запросов', 'Запросы, итого')
    df = row_number(df, 'Рейтинг постоянного\nпадения запросов', 'Запросы, итого')
    df = row_number(df, 'Рейтинг по запросам ВНВН', 'Запросы 2023')
    df = row_number(df, 'Рейтинг по запросам НВНВ', 'Запросы 2023')
    df = row_number(df, 'Рейтинг стабильности\nчисла запросов', 'Запросы, итого', filter_df=False)

    df = row_number(df, 'Рейтинг схожести динамики\nпользователей и запросов', 'Запросы, итого', filter_df=False)
    df = row_number(df, 'Антирейтинг схожести динамики\nпользователей и запросов', 'Запросы, итого', desc=False, filter_df=False)
    
    return df

In [957]:
def horizontal_optimization(table_to_optimize):
    new_svod = table_to_optimize
    try:
        test_svod = new_svod.drop(['InLists', 'inList, сумма'], axis=1).copy()
    except:
        test_svod = new_svod.copy()
    index_set = set(new_svod.index)
    cnt = 0
    total_sum = 9999
    while cnt < 10000 and len(index_set)> 0:
        j = index_set.pop()
        cnt += 1
        cols = [i for i in test_svod.loc[j][test_svod.loc[j] > 0].sort_values(ascending=False).index]
        for i in cols:   
            cnt_horizontal = test_svod.loc[j].drop('cnt_horizontal').replace(range(1, 31), 1).sum()
            cnt_vertical = test_svod[i].replace(range(1, 31), 1).sum()
            if cnt_vertical > 15 and cnt_horizontal > 1:
                test_svod[i][j] = 0
        
        cols_set = set(cols)
        cnt2 = 0
        while cols_set:
            i = cols_set.pop()
            cnt_horizontal = test_svod.loc[j].drop('cnt_horizontal').replace(range(1, 31), 1).sum()
            cnt_vertical = test_svod[i].replace(range(1, 31), 1).sum()
            if cnt_vertical > 15 and cnt_horizontal > 1:
                test_svod[i][j] = 0
        total_sum = test_svod.sum(axis=1).sum()
        
        #print(total_sum)
    test_svod['cnt_horizontal'] = test_svod.drop('cnt_horizontal', axis=1).replace(range(1, 31), 1).sum(axis=1)
    test_svod = test_svod.sort_values('cnt_horizontal', ascending=False)
    return test_svod

In [958]:
def vertical_optimization(table_to_optimize, original_table):
    df = table_to_optimize
    new_svod = original_table
    for i in range(10):
        for cat_to_optimize in df.index:
            # сейчас есть значение, превращаем в ноль
            col = df.drop('cnt_horizontal', axis=1).loc[cat_to_optimize].sort_values(ascending=False).index[0]

            # сейчас 0 возвращаем значение из бэкапа
            try:
                cat_to_reset = (
                     df
                     .loc[df.index != cat_to_optimize]
                     .loc[(new_svod.sort_index().loc[:, col] != df.sort_index().loc[:, col]) 
                          & (df[col] == 0)]
                     .loc[:,[col, 'cnt_horizontal']]
                     .sort_values([col, 'cnt_horizontal'], ascending=[False, True])
                     .index[0]
                )
                if df.loc[cat_to_reset, 'cnt_horizontal'] + 1 < df.loc[cat_to_optimize, 'cnt_horizontal']:
                    df.loc[cat_to_optimize, col] = 0
                    df.loc[cat_to_reset, col] = new_svod.loc[cat_to_reset, col]
                    df['cnt_horizontal'] = df.drop('cnt_horizontal', axis=1).replace(range(1, 31), 1).sum(axis=1)
            except:
                pass
    return df

In [845]:
original_svod = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Мастер_таблица_2ГИС_Москва_230315_1745.xlsx")

original_svod_sorted = (
    original_svod
    .fillna(0)
    .set_index('Рубрика')
).sort_values('Рубрика')

original_svod_counted = count_ratings(original_svod_sorted)

new_svod = original_svod_counted[[
 'Рейтинг по пользователям, итого',
 'Рейтинг по пользователям 2020',
 'Рейтинг по пользователям 2023',
 'Антирейтинг по пользователям 2023',
 'Антирейтинг по пользователям 2020',
 'Рейтинг постоянного\nроста пользователей',
 'Рейтинг постоянного\nпадения пользователей',
 'Рейтинг по пользователям 2021',
 'Рейтинг по пользователям 2022',
 'Антирейтинг по пользователям 2021',
 'Антирейтинг по пользователям 2022',
 'Рейтинг по пользователям ВНВН',
 'Рейтинг по пользователям НВНВ',
 'Рейтинг стабильности\nчисла пользователей',
 'Рейтинг схожести динамики\nпользователей и запросов',
 'Рейтинг по запросам, итого',
 'Рейтинг по запросам 2020',
 'Рейтинг по запросам 2023',
 'Антирейтинг по запросам 2023',
 'Антирейтинг по запросам 2020',
 'Рейтинг постоянного\nроста запросов',
 'Рейтинг постоянного\nпадения запросов',
 'Рейтинг по запросам 2021',
 'Рейтинг по запросам 2022',
 'Антирейтинг по запросам 2021',
 'Антирейтинг по запросам 2022',
 'Рейтинг по запросам ВНВН',
 'Рейтинг по запросам НВНВ',
 'Рейтинг стабильности\nчисла запросов',
 'Антирейтинг схожести динамики\nпользователей и запросов'
            ]]

new_svod['cnt_horizontal'] = new_svod.replace(range(1, 31), 1).sum(axis=1)
new_svod = new_svod.sort_values('cnt_horizontal', ascending=False)

C:\Users\Mamed\Anaconda\lib\site-packages\scipy\stats\stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [846]:
new_svod.value_counts('cnt_horizontal').sort_index()

cnt_horizontal
0.0    1198
1.0     140
2.0     112
3.0      26
4.0      22
5.0      14
6.0      10
7.0       5
9.0      20
dtype: int64

In [847]:
hor_optimized_svod = horizontal_optimization(new_svod)

hor_optimized_svod.value_counts('cnt_horizontal').sort_index()

cnt_horizontal
0.0    1198
1.0     308
2.0      17
3.0       5
4.0       4
5.0       2
6.0       4
7.0       1
8.0       2
9.0       6
dtype: int64

In [848]:
vert_optimized_svod = vertical_optimization(hor_optimized_svod, new_svod)

vert_optimized_svod.value_counts('cnt_horizontal').sort_index()

cnt_horizontal
0.0    1198
1.0     274
2.0      36
3.0      18
4.0      21
dtype: int64

In [959]:
optimized_svod = vert_optimized_svod.reset_index()
optimized_svod['В рейтингах, раз'] = optimized_svod['cnt_horizontal']
optimized_svod = optimized_svod.drop('cnt_horizontal', axis=1).sort_values('Рубрика')

final_svod = original_svod_sorted.copy().reset_index()

final_svod = final_svod[['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
       'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
       'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
       'Запросы 2023', 'Запросы, итого', 'Запросы, %',
       'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
       'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
       'Запросов на пользователя, итого']].sort_values('№')

final_svod = final_svod.merge(optimized_svod,left_on='Рубрика', right_on= 'Рубрика')

final_svod['Рейтинг суммарный'] = final_svod[new_svod.drop('cnt_horizontal', axis=1).columns].sum(axis=1)

final_svod['Рейтинг средний'] = final_svod[new_svod.drop('cnt_horizontal', axis=1).columns].mean(axis=1)

final_svod = final_svod.sort_index().sort_values('№')

for i in new_svod.drop('cnt_horizontal', axis=1).columns:
    final_svod = row_number(final_svod, i, i, desc=False, filter_df=False, index=15, filter_zero=True)

final_svod.to_excel('final_svod_master.xlsx', index=False)